<a href="https://colab.research.google.com/github/13194307/UTS_ML2019_ID13194307/blob/master/ML_A2_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

In [0]:
# Environment and constant preparation
%matplotlib inline
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
try:
    import plotly.graph_objects as go
except:
    !pip install plotly==4.1.0
import matplotlib.pyplot as plt
import torchvision.datasets as cvdata
import torchvision.transforms as transforms
from pathlib import Path
import torch

DATA_FOLDER = Path("./data").absolute()
DATA_FOLDER.mkdir(parents=True, exist_ok=True)
DATA_FOLDER = str(DATA_FOLDER)
COLAB = True
IRIS_EXPERIMENT_X_SPACE_QUANTISATION_BIN_NUM = 50 # self-interpretation

In [0]:
class MLP:
    def __init__(self, neuronsPerLayer):
        w0 = random.random()
        w1 = random.random()
        self.layers = [[tuple(random.random() for _ in range(0, 2)) #creates random weights
                        for _ in range(0, neuronsPerLayer[0])]] #Input layer
        
        for i in range(1, len(neuronsPerLayer)):
          layer = [tuple(random.random() for _ in range(0, neuronsPerLayer[i-1])) #creates random weights 
                   for _ in range(0, neuronsPerLayer[i])] #Hidden layers
            
          self.layers.append(layer)
          
        outputLayer = [tuple(random.random() #creates random weights
                             for _ in range(0, neuronsPerLayer[-1]))] #Output layer
        self.layers.append(outputLayer)
        
    def feedForward(self, x):
        inputWeights = self.getWeights(0)
        #print(inputWeights)
        lastLayerOutput = [(np.maximum(np.dot(inputWeights[i], x), 0))
                          for i in range(0, len(inputWeights))] #relu activation
        '''lastLayerOutput = [(np.maximum(np.dot(inputWeights[i][0], x[0]), 0), 
                           np.maximum(np.dot(inputWeights[i][1], x[1]), 0))
                          for i in range(0, len(inputWeights))] #relu activation'''
        
        for i in range(1, len(self.layers) - 1):
          layerWeights = self.getWeights(i)
          #print(layerWeights)
          #print(lastLayerOutput)
          currLayerOutput = np.maximum(np.dot(layerWeights, lastLayerOutput), 0)
          lastLayerOutput = currLayerOutput
          
        outputWeights = self.getWeights(-1)
        lastLayerOutput = np.dot(outputWeights, lastLayerOutput)
        
        #print("Last Output:",lastLayerOutput)
        return lastLayerOutput[0]
    
    def predict(self, x):
        prediction = []
        
        for sampleX in x:
          prediction.append(self.feedForward(sampleX))
      
        '''prediction = x[:, 0] * self.w0 \
            + x[:, 1] * self.w1 \
            + self.b '''
        prediction = np.array(prediction)
        prediction[prediction > 0] = 1
        prediction[prediction <= 0] = -1
        return prediction
        
    def getWeights(self, layerNum):
        return self.layers[layerNum]
    
    def update(self, dw0, dw1, db, verbose=False):
        if verbose:
            print("Old perceptron:", self)
        self.w0 += dw0
        self.w1 += dw1
        self.b += db
        if verbose: 
            print("New perceptron:", self)
            
    def set_param(self, w0, w1, b, verbose=False):
        if verbose:
            print("Old perceptron:", self)
        self.w0 = w0
        self.w1 = w1
        self.b = b
        if verbose: 
            print("New perceptron:", self)
            
            
    def fit(self, X, y):
        raise NotImplementedError
    
    def __str__(self):
        output = ""
        for i in range(0, len(self.layers)):
          output+="Layer {}:\n".format(str(i+1)) 
          
          for neuron in self.layers[i]:
           #output+= ("\tW0:{:.2f}, W1:{:.2f}, b:{:.2f}\n".format(neuron[0], neuron[1], neuron[2]))
            output+="\t"
            for j in range(0, len(neuron)):
              output+= ("W{}:{:.2f}, ".format(j, neuron[j]))
            
            output+="\n"
        
        return output

In [132]:
mlp = MLP([3, 3, 3])

print(mlp)

Layer 1:
	W0:0.72, W1:0.81, 
	W0:0.60, W1:0.04, 
	W0:0.44, W1:0.06, 
Layer 2:
	W0:0.18, W1:0.92, W2:0.74, 
	W0:0.47, W1:0.27, W2:0.80, 
	W0:0.03, W1:0.23, W2:0.91, 
Layer 3:
	W0:0.96, W1:0.69, W2:0.00, 
	W0:0.89, W1:0.80, W2:0.54, 
	W0:0.36, W1:0.23, W2:0.59, 
Layer 4:
	W0:0.42, W1:0.56, W2:0.26, 



In [7]:
# Download and make dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_trainset = cvdata.CIFAR10(
    root=DATA_FOLDER, train=True,
    download=True, transform=transform)

cifar_all_trainloader = torch.utils.data.DataLoader(
    cifar_trainset, batch_size=64,
    shuffle=False, num_workers=6)

# take two classes for a subset
cifar_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# collect samples in an overall array
images = []
labels = []
indexes = []
for i, (x, y) in enumerate(cifar_trainset):
    if cifar_classes[y] in ["plane", "bird"]:
        images.append(x)
        twoclass_label = 1 if cifar_classes[y] == "plane" else -1
        labels.append(twoclass_label)
        indexes.append(i)
        
X = torch.stack(images).numpy()
y = np.array(labels)
indexes = np.array(indexes)

0it [00:00, ?it/s]

170500096it [00:04, 38921726.37it/s]                               


In [8]:
# Defining processing functions
def feature_maker_overall_green(X):
    return X[:, 1].reshape(X.shape[0], -1).sum(axis=1)

def feature_maker_overall_blue(X):
    return X[:, 2].reshape(X.shape[0], -1).sum(axis=1)

def take_a_separable_subset(X, y, original_indexes):
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    lr.fit(X, y)
    pred = lr.predict(X)
    pred_prob = lr.predict_log_proba(X_train)
    confident_ind = np.logical_or(pred_prob[:, 0] > np.log(0.55),
                                  pred_prob[:, 1] > np.log(0.55))
    ind = np.logical_and(confident_ind, pred==y)
    X_simple = X[ind]
    y_simple = y[ind]
    original_indexes_simple = original_indexes[ind]
    return X_simple, y_simple, original_indexes_simple

def prepare_cifar_two_class_data(
    X, y, original_indexes,
    simple=False,
    train_size=1000,
    make_feature_1=feature_maker_overall_green, 
    make_feature_2=feature_maker_overall_blue):
    feature1 = make_feature_1(X)
    feature2 = make_feature_2(X)
    X = np.stack([feature1, feature2], axis=1)
    # normalise to -1 to +1
    X -= X.min(axis=0)
    X /= X.max(axis=0)
    X -= 0.5
    X *= 2.0
    
    X_train, X_test, y_train, y_test, ind_train, ind_test = \
        train_test_split(X, y, original_indexes, 
                         train_size=train_size, test_size=len(y)-train_size)
    return X_train, X_test, y_train, y_test, ind_train, ind_test

def prepare_cifar_two_class_separable_data(
    X, y, original_indexes, train_size=100):
    
    X_simple, y_simple, indexes_simple = \
        take_a_separable_subset(X, y, original_indexes)
    
    X_train_simple, X_test_simple, \
    y_train_simple, y_test_simple, \
    ind_train_simple, ind_test_simple = \
        train_test_split(X_simple, y_simple, indexes_simple, 
                         train_size=train_size, test_size=len(y_simple)-100)
    return X_train_simple, X_test_simple, \
        y_train_simple, y_test_simple, \
        ind_train_simple, ind_test_simple
    
X_train, X_test, y_train, y_test, ind_train, ind_test = \
    prepare_cifar_two_class_data(X, y, indexes)
X_train_simple, X_test_simple, \
y_train_simple, y_test_simple, \
ind_train_simple, ind_test_simple = prepare_cifar_two_class_separable_data(
    X_train, y_train, ind_train)

def quick_separable_train_sample(n=100):
    X_train_simple, X_test_simple, \
    y_train_simple, y_test_simple, \
    ind_train_simple, ind_test_simple = prepare_cifar_two_class_separable_data(
        X_train, y_train, ind_train, train_size=n)
    
    return X_train_simple, y_train_simple, ind_train_simple

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [0]:
# Defining vis-functions
def show_cifar_image(img_id):
    npimg = ((cifar_trainset[img_id][0].detach().numpy() + 1.0) \
        * 128).astype(np.uint8).transpose((1, 2, 0))
    # plt.imshow(npimg, interpolation='nearest') # for larger view
    return Image.fromarray(npimg), npimg

def encode_sample_image(index):
    """
    Generate the resource url to display an image in a webpage.
    This is not used in notebooks. But you can take the function
    in a standalone Python program as a web-server to visual data models.
    """
    import base64
    from io import BytesIO

    pil_img = Image.fromarray(((cifar_trainset[index][0].numpy()
                                .transpose([1, 2, 0]) + 1.0) * 128).astype(np.uint8))
    buff = BytesIO()
    pil_img.save(buff, format="JPEG")
    new_image_string = base64.b64encode(buff.getvalue()).decode("utf-8")
    # print(new_image_string[:100])
    return """<img src="data:image/png;base64,""" \
        + """"></img>"""

def show_perceptron_model(model, X_train, y_train, indexes_train=[]):
    layout = go.Layout(
        xaxis=go.layout.XAxis(
            range=[-1, 1],
            showgrid=True,
            zeroline=True,
            showline=True,
            gridcolor='#bdbdbd',
            gridwidth=1,
            zerolinecolor='#969696',
            zerolinewidth=2,
            linecolor='#636363',
            linewidth=2,
            mirror=True,
        ),
        yaxis=go.layout.YAxis(
            range=[-1, 1],
            showgrid=True,
            zeroline=True,
            showline=True,
            gridcolor='#bdbdbd',
            gridwidth=1,
            zerolinecolor='#969696',
            zerolinewidth=2,
            linecolor='#636363',
            linewidth=2,
            mirror=True,
       ),
       height=600,
       width=600,
    )

    # visualise perceptron model on a grid
    x_grid, y_grid = np.meshgrid(np.arange(-1, 1.01, 0.05), np.arange(-1, 1.01, 0.05))
    grid_X = np.stack(( x_grid.flatten(), y_grid.flatten()) ).T
    grid_pred = model.predict(grid_X)
    train_pred = np.array(model.predict(X_train))
    train_error_num = (train_pred.astype(np.int) != y_train.astype(np.int)).sum()
    E_in = train_error_num / len(y_train)

    scatter_grid = go.Scatter(
        x=grid_X[:, 0], y=grid_X[:, 1], 
        marker=dict(
            size=6,
            cmax=1,
            cmin=-1,
            line_width=1,
            color=grid_pred,
            colorscale="Cividis",
            symbol="square",
            opacity=0.5
        ),
        mode="markers",
    #     colorscale=,
        name="'All' X Space Samples",
        hoverinfo="none")
    
    contour_grid = go.Contour(
        z=grid_pred,
        x=grid_X[:, 0], # horizontal axis
        y=grid_X[:, 1], # vertical axis
        hoverinfo="none",
        colorscale="Cividis",
        showscale=False
    )
    
    if len(indexes_train) == 0:
        scatter_train_text = ["X:({:.02f}, {:.02f})<br>y:{}, pred:{}".format(x0, x1, int(y), int(p)) 
              for (x0, x1), y, p in zip(X_train, y_train, train_pred)] 
    else:
        scatter_train_text = ["X:({:.02f}, {:.02f})<br>y:{}, pred:{}, ImgID {:d}"\
                              .format(x0, x1, int(y), int(p), i) 
                              for (x0, x1), y, p, i in zip(X_train, y_train, train_pred, indexes_train)] 
        

    scatter_train = go.Scatter(
        x=X_train[:, 0], y=X_train[:, 1],
         marker=dict(
             size=12,
             cmax=1,
             cmin=-1,
             color=y_train,
             colorscale="Cividis",
             line=dict(
                 width=2,
                 color=["green" if prediction == ground_truth else "red"
                        for prediction, ground_truth in zip(train_pred, y_train)]
             )

        ),
        mode="markers",
        name="Dataset Samples",
        text=scatter_train_text,
        hoverinfo="text")

    fig = go.Figure(
        data=[
            contour_grid, scatter_train
        ],
        layout=layout,
        layout_title_text="Two Object Class 2D Data<br>#.errors={:d}, E_in={:.3f}"\
            .format(train_error_num, E_in)
    )
    if COLAB:
        fig.show(renderer="colab")
    else:
        fig.show()

In [141]:
import sklearn
from sklearn.metrics import *

X_easy, y_easy, ind_easy = quick_separable_train_sample(20)
show_perceptron_model(mlp, X_easy, y_easy, ind_easy)
#predictions = mlp.predict(X_easy)
#f1_score(predictions, y_easy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [81]:
x = [1, 2, 3]
y = [4, 5, 6]
print(np.dot(x, y))

32
